In [129]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [23]:
import torch
import torch.nn as nn
import numpy as np
from utilis_ForsaV2 import *
import utilis_ForsaV2
# !pip install fxpmath\
from fxpmath import Fxp
import time
word_size  = 8
frac_size = 6
utilis_ForsaV2.word_size = word_size
utilis_ForsaV2.frac_size = frac_size

# from tensorboardX import SummaryWriter
# from transforms import *
# from models_cust import *


In [97]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.eval()

Using cache found in C:\Users\huruy/.cache\torch\hub\pytorch_vision_v0.10.0


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [98]:
from torchsummary import summary
summary(model.to('cuda'), (3, 224, 224)) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
       BasicConv2d-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
       BasicConv2d-7           [-1, 64, 56, 56]               0
            Conv2d-8          [-1, 192, 56, 56]         110,592
       BatchNorm2d-9          [-1, 192, 56, 56]             384
      BasicConv2d-10          [-1, 192, 56, 56]               0
        MaxPool2d-11          [-1, 192, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          12,288
      BatchNorm2d-13           [-1, 64, 28, 28]             128
      BasicConv2d-14           [-1, 64,

# Demonstrating sorting for first regular convs

In [99]:
No_conv_layers=3 # to exlude classifier network
sw_wt = np.zeros(No_conv_layers)
sw_bn = np.zeros((No_conv_layers, 4))
sw_wt_sorted = np.zeros(No_conv_layers)
sw_bn_sorted = np.zeros((No_conv_layers, 4))
########## sorting conv1.conv ##########
_, new_indx1 = sortFullMatrix_V2(model.conv1.conv.weight)
model.conv1.conv.weight.data, _, sw_wt[0], sw_wt_sorted[0]  = compare_sw_sort(model.conv1.conv.weight.data, new_indx1, -3)
model.conv1.bn.weight.data, _, sw_bn[0][0], sw_bn_sorted[0][0]  = compare_sw_sort(model.conv1.bn.weight.data, new_indx1, -1)
model.conv1.bn.bias.data, _, sw_bn[0][1], sw_bn_sorted[0][1]  = compare_sw_sort(model.conv1.bn.bias.data, new_indx1, -1)
model.conv1.bn.running_mean.data, _, sw_bn[0][2], sw_bn_sorted[0][2]  = compare_sw_sort(model.conv1.bn.running_mean.data, new_indx1, -1)
model.conv1.bn.running_var.data, _, sw_bn[0][3], sw_bn_sorted[0][3]  = compare_sw_sort(model.conv1.bn.running_var.data, new_indx1, -1)
# plot_dist(model.conv1.bn.weight)
########## rearranging conv2.conv ##########
model.conv2.conv.weight.data = model.conv2.conv.weight[:, new_indx1, :, :]


Switching before sorting 35957.0
Switching after sorting 31893.0
Percentage of reduction 11.302388964596602 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 244.0
Switching after sorting 251.0
Percentage of reduction -2.8688524590163933 %
Switching before sorting 249.0
Switching after sorting 263.0
Percentage of reduction -5.622489959839357 %
Switching before sorting 220.0
Switching after sorting 203.0
Percentage of reduction 7.7272727272727275 %


In [100]:
########## sorting conv2.conv ##########
_, new_indx1 = sortFullMatrix_V2(model.conv2.conv.weight)
model.conv2.conv.weight.data, _, sw_wt[1], sw_wt_sorted[1]  = compare_sw_sort(model.conv2.conv.weight.data, new_indx1, -3)
model.conv2.bn.weight.data, _, sw_bn[1][0], sw_bn_sorted[1][0]  = compare_sw_sort(model.conv2.bn.weight.data, new_indx1, -1)
model.conv2.bn.bias.data, _, sw_bn[1][1], sw_bn_sorted[1][1]  = compare_sw_sort(model.conv2.bn.bias.data, new_indx1, -1)
model.conv2.bn.running_mean.data, _, sw_bn[1][2], sw_bn_sorted[1][2]  = compare_sw_sort(model.conv2.bn.running_mean.data, new_indx1, -1)
model.conv2.bn.running_var.data, _, sw_bn[1][3], sw_bn_sorted[1][3]  = compare_sw_sort(model.conv2.bn.running_var.data, new_indx1, -1)
# plot_dist(model.conv1.bn.weight)
########## rearranging conv3.conv ##########
model.conv3.conv.weight.data = model.conv3.conv.weight[:, new_indx1, :, :]


Switching before sorting 15587.0
Switching after sorting 13101.0
Percentage of reduction 15.949188426252647 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 243.0
Switching after sorting 243.0
Percentage of reduction 0.0 %
Switching before sorting 251.0
Switching after sorting 260.0
Percentage of reduction -3.585657370517928 %
Switching before sorting 260.0
Switching after sorting 262.0
Percentage of reduction -0.7692307692307693 %


In [101]:
########## sorting conv3.conv ##########
_, new_indx1 = sortFullMatrix_V2(model.conv3.conv.weight)
model.conv3.conv.weight.data, _, sw_wt[2], sw_wt_sorted[2]  = compare_sw_sort(model.conv3.conv.weight.data, new_indx1, -3)
model.conv3.bn.weight.data, _, sw_bn[2][0], sw_bn_sorted[2][0]  = compare_sw_sort(model.conv3.bn.weight.data, new_indx1, -1)
model.conv3.bn.bias.data, _, sw_bn[2][1], sw_bn_sorted[2][1]  = compare_sw_sort(model.conv3.bn.bias.data, new_indx1, -1)
model.conv3.bn.running_mean.data, _, sw_bn[2][2], sw_bn_sorted[2][2]  = compare_sw_sort(model.conv3.bn.running_mean.data, new_indx1, -1)
model.conv3.bn.running_var.data, _, sw_bn[2][3], sw_bn_sorted[2][3]  = compare_sw_sort(model.conv3.bn.running_var.data, new_indx1, -1)
# plot_dist(model.conv1.bn.weight)
########## rearranging inception3a.branch1.conv ##########
model.inception3a.branch1.conv.weight.data = model.inception3a.branch1.conv.weight[:, new_indx1, :, :]
########## rearranging inception3a.branch2.0.conv ##########
model.inception3a.branch2[0].conv.weight.data = model.inception3a.branch2[0].conv.weight[:, new_indx1, :, :]
########## rearranging inception3a.branch3.0.conv ##########
model.inception3a.branch3[0].conv.weight.data = model.inception3a.branch3[0].conv.weight[:, new_indx1, :, :]
########## rearranging inception3a.branch4.1.conv ##########
model.inception3a.branch4[1].conv.weight.data = model.inception3a.branch4[1].conv.weight[:, new_indx1, :, :]


Switching before sorting 415246.0
Switching after sorting 369064.0
Percentage of reduction 11.121600208069433 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 724.0
Switching after sorting 682.0
Percentage of reduction 5.801104972375691 %
Switching before sorting 757.0
Switching after sorting 737.0
Percentage of reduction 2.642007926023778 %
Switching before sorting 677.0
Switching after sorting 689.0
Percentage of reduction -1.7725258493353029 %


In [102]:
print('sw_wt before sorting', sw_wt)
print('sw_wt_sorted after sorting', sw_wt_sorted)
print('sw_bn before sorting', sw_bn)
print('sw_bn_sorted after sorting', sw_bn_sorted)


sw_wt before sorting [ 35957.  15587. 415246.]
sw_wt_sorted after sorting [ 31893.  13101. 369064.]
sw_bn before sorting [[  0. 244. 249. 220.]
 [  0. 243. 251. 260.]
 [  0. 724. 757. 677.]]
sw_bn_sorted after sorting [[  0. 251. 263. 203.]
 [  0. 243. 260. 262.]
 [  0. 682. 737. 689.]]


# Demonstrating sorting inside the first inception module inception3a

In [104]:
No_inc_layers=9 # to exlude classifier network
sw_wt = np.zeros((No_inc_layers, 6))
# sw_bn = np.zeros((No_inc_layers, 6, 4))
sw_wt_sorted = np.zeros((No_inc_layers, 6))
# sw_bn_sorted = np.zeros((No_inc_layers, 4))

In [105]:
# Inception 1
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception3a.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception3a.branch1.conv.weight)
model.inception3a.branch1.conv.weight.data, _, sw_wt[0][0], sw_wt_sorted[0][0]  = compare_sw_sort(model.inception3a.branch1.conv.weight.data, new_indxb1, -3)
model.inception3a.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception3a.branch1.bn.weight.data, new_indxb1, -1)
model.inception3a.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception3a.branch1.bn.bias.data, new_indxb1, -1)
model.inception3a.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3a.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception3a.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3a.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception3a.branch2[0].conv.weight)
model.inception3a.branch2[0].conv.weight.data, _, sw_wt[0][1], sw_wt_sorted[0][1]  = compare_sw_sort(model.inception3a.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception3a.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3a.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception3a.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3a.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception3a.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3a.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception3a.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3a.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception3a.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception3a.branch2[1].conv.weight)
model.inception3a.branch2[1].conv.weight.data, _, sw_wt[0][2], sw_wt_sorted[0][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception3a.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3a.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception3a.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3a.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception3a.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3a.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception3a.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3a.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception3a.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception3a.branch3[0].conv.weight)
model.inception3a.branch3[0].conv.weight.data, _, sw_wt[0][3], sw_wt_sorted[0][3]  = compare_sw_sort(model.inception3a.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception3a.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3a.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception3a.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3a.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception3a.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3a.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception3a.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3a.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception3a.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception3a.branch3[1].conv.weight)
model.inception3a.branch3[1].conv.weight.data, _, sw_wt[0][4], sw_wt_sorted[0][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception3a.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3a.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception3a.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3a.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception3a.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3a.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception3a.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3a.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception3a.branch1.conv.weight.shape[0] + model.inception3a.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception3a.branch4[1].conv.weight)
model.inception3a.branch4[1].conv.weight.data, _, sw_wt[0][5], sw_wt_sorted[0][5]  = compare_sw_sort(model.inception3a.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception3a.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3a.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception3a.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3a.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception3a.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3a.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception3a.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3a.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception3a.branch1.conv.weight.shape[0] + model.inception3a.branch2[1].conv.weight.shape[0] + model.inception3a.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate
# Rearranging all branchs of inception 2 along channel dimension to correct Branch 1 sortng 
model.inception3b.branch1.conv.weight.data = model.inception3b.branch1.conv.weight[:, new_indxIncp2, :, :]
model.inception3b.branch2[0].conv.weight.data =model.inception3b.branch2[0].conv.weight[:, new_indxIncp2, :, :]
model.inception3b.branch3[0].conv.weight.data = model.inception3b.branch3[0].conv.weight[:, new_indxIncp2, :, :]
model.inception3b.branch4[1].conv.weight.data = model.inception3b.branch4[1].conv.weight[:, new_indxIncp2, :, :]


Switching before sorting 46409.0
Switching after sorting 41392.0
Percentage of reduction 10.810403154560538 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 278.0
Switching after sorting 266.0
Percentage of reduction 4.316546762589928 %
Switching before sorting 284.0
Switching after sorting 273.0
Percentage of reduction 3.8732394366197185 %
Switching before sorting 234.0
Switching after sorting 252.0
Percentage of reduction -7.6923076923076925 %
Switching before sorting 71234.0
Switching after sorting 62260.0
Percentage of reduction 12.597916725159333 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 391.0
Switching after sorting 389.0
Percentage of reduction 0.5115089514066496 %
Switching before sorting 390.0
Switching after sorting 385.0
Percentage of reduction 1.2820512820512822 %
Switching before sorting 335.0
Switching after sorting 329.0
Percentage of r

In [106]:
## Inception 2
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception3b.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception3b.branch1.conv.weight)
model.inception3b.branch1.conv.weight.data, _, sw_wt[1][0], sw_wt_sorted[1][0]  = compare_sw_sort(model.inception3b.branch1.conv.weight.data, new_indxb1, -3)
model.inception3b.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception3b.branch1.bn.weight.data, new_indxb1, -1)
model.inception3b.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception3b.branch1.bn.bias.data, new_indxb1, -1)
model.inception3b.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3b.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception3b.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3b.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception3b.branch2[0].conv.weight)
model.inception3b.branch2[0].conv.weight.data, _, sw_wt[1][1], sw_wt_sorted[1][1]  = compare_sw_sort(model.inception3b.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception3b.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3b.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception3b.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3b.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception3b.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3b.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception3b.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3b.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception3b.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception3b.branch2[1].conv.weight)
model.inception3b.branch2[1].conv.weight.data, _, sw_wt[1][2], sw_wt_sorted[1][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception3b.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3b.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception3b.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3b.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception3b.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3b.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception3b.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3b.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception3b.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception3b.branch3[0].conv.weight)
model.inception3b.branch3[0].conv.weight.data, _, sw_wt[1][3], sw_wt_sorted[1][3]  = compare_sw_sort(model.inception3b.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception3b.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3b.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception3b.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3b.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception3b.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3b.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception3b.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3b.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception3b.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception3b.branch3[1].conv.weight)
model.inception3b.branch3[1].conv.weight.data, _, sw_wt[1][4], sw_wt_sorted[1][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception3b.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3b.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception3b.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3b.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception3b.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3b.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception3b.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3b.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception3b.branch1.conv.weight.shape[0] + model.inception3b.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception3b.branch4[1].conv.weight)
model.inception3b.branch4[1].conv.weight.data, _, sw_wt[1][5], sw_wt_sorted[1][5]  = compare_sw_sort(model.inception3b.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception3b.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception3b.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception3b.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception3b.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception3b.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception3b.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception3b.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception3b.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception3b.branch1.conv.weight.shape[0] + model.inception3b.branch2[1].conv.weight.shape[0] + model.inception3b.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate

# Rearranging all branchs of inception 2 along channel dimension to correct Branch 1 sortng 
model.inception4a.branch1.conv.weight.data = model.inception4a.branch1.conv.weight[:, new_indxIncp2, :, :]
model.inception4a.branch2[0].conv.weight.data =model.inception4a.branch2[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4a.branch3[0].conv.weight.data = model.inception4a.branch3[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4a.branch4[1].conv.weight.data = model.inception4a.branch4[1].conv.weight[:, new_indxIncp2, :, :]


Switching before sorting 122703.0
Switching after sorting 107634.0
Percentage of reduction 12.280873328280482 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 481.0
Switching after sorting 457.0
Percentage of reduction 4.98960498960499 %
Switching before sorting 493.0
Switching after sorting 467.0
Percentage of reduction 5.273833671399594 %
Switching before sorting 391.0
Switching after sorting 427.0
Percentage of reduction -9.207161125319693 %
Switching before sorting 122718.0
Switching after sorting 108056.0
Percentage of reduction 11.94771753125051 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 466.0
Switching after sorting 441.0
Percentage of reduction 5.364806866952789 %
Switching before sorting 490.0
Switching after sorting 490.0
Percentage of reduction 0.0 %
Switching before sorting 374.0
Switching after sorting 354.0
Percentage of reduction 5.34759

In [107]:
## Inception 3
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception4a.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception4a.branch1.conv.weight)
model.inception4a.branch1.conv.weight.data, _, sw_wt[2][0], sw_wt_sorted[2][0]  = compare_sw_sort(model.inception4a.branch1.conv.weight.data, new_indxb1, -3)
model.inception4a.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception4a.branch1.bn.weight.data, new_indxb1, -1)
model.inception4a.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception4a.branch1.bn.bias.data, new_indxb1, -1)
model.inception4a.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4a.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception4a.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4a.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception4a.branch2[0].conv.weight)
model.inception4a.branch2[0].conv.weight.data, _, sw_wt[2][1], sw_wt_sorted[2][1]  = compare_sw_sort(model.inception4a.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception4a.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4a.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception4a.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4a.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception4a.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4a.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception4a.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4a.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception4a.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception4a.branch2[1].conv.weight)
model.inception4a.branch2[1].conv.weight.data, _, sw_wt[2][2], sw_wt_sorted[2][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception4a.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4a.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception4a.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4a.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception4a.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4a.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception4a.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4a.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception4a.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception4a.branch3[0].conv.weight)
model.inception4a.branch3[0].conv.weight.data, _, sw_wt[2][3], sw_wt_sorted[2][3]  = compare_sw_sort(model.inception4a.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception4a.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4a.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception4a.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4a.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception4a.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4a.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception4a.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4a.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception4a.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception4a.branch3[1].conv.weight)
model.inception4a.branch3[1].conv.weight.data, _, sw_wt[2][4], sw_wt_sorted[2][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception4a.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4a.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception4a.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4a.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception4a.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4a.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception4a.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4a.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception4a.branch1.conv.weight.shape[0] + model.inception4a.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception4a.branch4[1].conv.weight)
model.inception4a.branch4[1].conv.weight.data, _, sw_wt[2][5], sw_wt_sorted[2][5]  = compare_sw_sort(model.inception4a.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception4a.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4a.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception4a.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4a.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception4a.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4a.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception4a.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4a.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception4a.branch1.conv.weight.shape[0] + model.inception4a.branch2[1].conv.weight.shape[0] + model.inception4a.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate

# Rearranging all branchs of inception 2 along channel dimension to correct Branch 1 sortng 
model.inception4b.branch1.conv.weight.data = model.inception4b.branch1.conv.weight[:, new_indxIncp2, :, :]
model.inception4b.branch2[0].conv.weight.data =model.inception4b.branch2[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4b.branch3[0].conv.weight.data = model.inception4b.branch3[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4b.branch4[1].conv.weight.data = model.inception4b.branch4[1].conv.weight[:, new_indxIncp2, :, :]


Switching before sorting 339103.0
Switching after sorting 304019.0
Percentage of reduction 10.346119025782727 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 730.0
Switching after sorting 693.0
Percentage of reduction 5.068493150684931 %
Switching before sorting 765.0
Switching after sorting 735.0
Percentage of reduction 3.9215686274509802 %
Switching before sorting 688.0
Switching after sorting 665.0
Percentage of reduction 3.3430232558139537 %
Switching before sorting 166546.0
Switching after sorting 151218.0
Percentage of reduction 9.203463307434582 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 362.0
Switching after sorting 364.0
Percentage of reduction -0.5524861878453039 %
Switching before sorting 390.0
Switching after sorting 385.0
Percentage of reduction 1.2820512820512822 %
Switching before sorting 333.0
Switching after sorting 348.0
Percentage o

In [108]:
## Inception 4
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception4b.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception4b.branch1.conv.weight)
model.inception4b.branch1.conv.weight.data, _, sw_wt[3][0], sw_wt_sorted[3][0]  = compare_sw_sort(model.inception4b.branch1.conv.weight.data, new_indxb1, -3)
model.inception4b.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception4b.branch1.bn.weight.data, new_indxb1, -1)
model.inception4b.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception4b.branch1.bn.bias.data, new_indxb1, -1)
model.inception4b.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4b.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception4b.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4b.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception4b.branch2[0].conv.weight)
model.inception4b.branch2[0].conv.weight.data, _, sw_wt[3][1], sw_wt_sorted[3][1]  = compare_sw_sort(model.inception4b.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception4b.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4b.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception4b.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4b.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception4b.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4b.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception4b.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4b.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception4b.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception4b.branch2[1].conv.weight)
model.inception4b.branch2[1].conv.weight.data, _, sw_wt[3][2], sw_wt_sorted[3][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception4b.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4b.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception4b.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4b.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception4b.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4b.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception4b.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4b.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception4b.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception4b.branch3[0].conv.weight)
model.inception4b.branch3[0].conv.weight.data, _, sw_wt[3][3], sw_wt_sorted[3][3]  = compare_sw_sort(model.inception4b.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception4b.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4b.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception4b.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4b.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception4b.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4b.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception4b.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4b.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception4b.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception4b.branch3[1].conv.weight)
model.inception4b.branch3[1].conv.weight.data, _, sw_wt[3][4], sw_wt_sorted[3][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception4b.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4b.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception4b.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4b.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception4b.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4b.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception4b.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4b.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception4b.branch1.conv.weight.shape[0] + model.inception4b.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception4b.branch4[1].conv.weight)
model.inception4b.branch4[1].conv.weight.data, _, sw_wt[3][5], sw_wt_sorted[3][5]  = compare_sw_sort(model.inception4b.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception4b.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4b.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception4b.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4b.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception4b.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4b.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception4b.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4b.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception4b.branch1.conv.weight.shape[0] + model.inception4b.branch2[1].conv.weight.shape[0] + model.inception4b.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate

# Rearranging all branchs of inception 2 along channel dimension to correct Branch 1 sortng 
model.inception4c.branch1.conv.weight.data = model.inception4c.branch1.conv.weight[:, new_indxIncp2, :, :]
model.inception4c.branch2[0].conv.weight.data =model.inception4c.branch2[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4c.branch3[0].conv.weight.data = model.inception4c.branch3[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4c.branch4[1].conv.weight.data = model.inception4c.branch4[1].conv.weight[:, new_indxIncp2, :, :]


Switching before sorting 301081.0
Switching after sorting 276999.0
Percentage of reduction 7.998512028324604 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 587.0
Switching after sorting 629.0
Percentage of reduction -7.155025553662692 %
Switching before sorting 645.0
Switching after sorting 620.0
Percentage of reduction 3.875968992248062 %
Switching before sorting 496.0
Switching after sorting 483.0
Percentage of reduction 2.620967741935484 %
Switching before sorting 213995.0
Switching after sorting 192303.0
Percentage of reduction 10.13668543657562 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 438.0
Switching after sorting 422.0
Percentage of reduction 3.65296803652968 %
Switching before sorting 424.0
Switching after sorting 430.0
Percentage of reduction -1.4150943396226414 %
Switching before sorting 331.0
Switching after sorting 299.0
Percentage of re

In [109]:
## Inception 5
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception4c.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception4c.branch1.conv.weight)
model.inception4c.branch1.conv.weight.data, _, sw_wt[4][0], sw_wt_sorted[4][0]  = compare_sw_sort(model.inception4c.branch1.conv.weight.data, new_indxb1, -3)
model.inception4c.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception4c.branch1.bn.weight.data, new_indxb1, -1)
model.inception4c.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception4c.branch1.bn.bias.data, new_indxb1, -1)
model.inception4c.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4c.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception4c.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4c.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception4c.branch2[0].conv.weight)
model.inception4c.branch2[0].conv.weight.data, _, sw_wt[4][1], sw_wt_sorted[4][1]  = compare_sw_sort(model.inception4c.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception4c.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4c.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception4c.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4c.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception4c.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4c.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception4c.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4c.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception4c.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception4c.branch2[1].conv.weight)
model.inception4c.branch2[1].conv.weight.data, _, sw_wt[4][2], sw_wt_sorted[4][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception4c.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4c.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception4c.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4c.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception4c.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4c.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception4c.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4c.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception4c.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception4c.branch3[0].conv.weight)
model.inception4c.branch3[0].conv.weight.data, _, sw_wt[4][3], sw_wt_sorted[4][3]  = compare_sw_sort(model.inception4c.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception4c.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4c.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception4c.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4c.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception4c.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4c.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception4c.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4c.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception4c.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception4c.branch3[1].conv.weight)
model.inception4c.branch3[1].conv.weight.data, _, sw_wt[4][4], sw_wt_sorted[4][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception4c.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4c.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception4c.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4c.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception4c.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4c.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception4c.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4c.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception4c.branch1.conv.weight.shape[0] + model.inception4c.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception4c.branch4[1].conv.weight)
model.inception4c.branch4[1].conv.weight.data, _, sw_wt[4][5], sw_wt_sorted[4][5]  = compare_sw_sort(model.inception4c.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception4c.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4c.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception4c.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4c.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception4c.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4c.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception4c.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4c.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception4c.branch1.conv.weight.shape[0] + model.inception4c.branch2[1].conv.weight.shape[0] + model.inception4c.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate

# Rearranging all branchs of inception 2 along channel dimension to correct Branch 1 sortng 
model.inception4d.branch1.conv.weight.data = model.inception4d.branch1.conv.weight[:, new_indxIncp2, :, :]
model.inception4d.branch2[0].conv.weight.data =model.inception4d.branch2[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4d.branch3[0].conv.weight.data = model.inception4d.branch3[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4d.branch4[1].conv.weight.data = model.inception4d.branch4[1].conv.weight[:, new_indxIncp2, :, :]


Switching before sorting 240730.0
Switching after sorting 222237.0
Percentage of reduction 7.682050429942259 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 517.0
Switching after sorting 492.0
Percentage of reduction 4.835589941972921 %
Switching before sorting 499.0
Switching after sorting 506.0
Percentage of reduction -1.402805611222445 %
Switching before sorting 430.0
Switching after sorting 416.0
Percentage of reduction 3.255813953488372 %
Switching before sorting 249343.0
Switching after sorting 223067.0
Percentage of reduction 10.538094111324561 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 427.0
Switching after sorting 410.0
Percentage of reduction 3.981264637002342 %
Switching before sorting 466.0
Switching after sorting 485.0
Percentage of reduction -4.07725321888412 %
Switching before sorting 376.0
Switching after sorting 370.0
Percentage of re

In [110]:
## Inception 6
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception4d.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception4d.branch1.conv.weight)
model.inception4d.branch1.conv.weight.data, _, sw_wt[5][0], sw_wt_sorted[5][0]  = compare_sw_sort(model.inception4d.branch1.conv.weight.data, new_indxb1, -3)
model.inception4d.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception4d.branch1.bn.weight.data, new_indxb1, -1)
model.inception4d.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception4d.branch1.bn.bias.data, new_indxb1, -1)
model.inception4d.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4d.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception4d.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4d.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception4d.branch2[0].conv.weight)
model.inception4d.branch2[0].conv.weight.data, _, sw_wt[5][1], sw_wt_sorted[5][1]  = compare_sw_sort(model.inception4d.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception4d.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4d.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception4d.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4d.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception4d.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4d.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception4d.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4d.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception4d.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception4d.branch2[1].conv.weight)
model.inception4d.branch2[1].conv.weight.data, _, sw_wt[5][2], sw_wt_sorted[5][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception4d.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4d.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception4d.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4d.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception4d.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4d.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception4d.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4d.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception4d.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception4d.branch3[0].conv.weight)
model.inception4d.branch3[0].conv.weight.data, _, sw_wt[5][3], sw_wt_sorted[5][3]  = compare_sw_sort(model.inception4d.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception4d.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4d.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception4d.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4d.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception4d.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4d.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception4d.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4d.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception4d.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception4d.branch3[1].conv.weight)
model.inception4d.branch3[1].conv.weight.data, _, sw_wt[5][4], sw_wt_sorted[5][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception4d.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4d.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception4d.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4d.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception4d.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4d.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception4d.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4d.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception4d.branch1.conv.weight.shape[0] + model.inception4d.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception4d.branch4[1].conv.weight)
model.inception4d.branch4[1].conv.weight.data, _, sw_wt[5][5], sw_wt_sorted[5][5]  = compare_sw_sort(model.inception4d.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception4d.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4d.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception4d.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4d.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception4d.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4d.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception4d.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4d.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception4d.branch1.conv.weight.shape[0] + model.inception4d.branch2[1].conv.weight.shape[0] + model.inception4d.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate

# Rearranging all branchs of inception 2 along channel dimension to correct Branch 1 sortng 
model.inception4e.branch1.conv.weight.data = model.inception4e.branch1.conv.weight[:, new_indxIncp2, :, :]
model.inception4e.branch2[0].conv.weight.data =model.inception4e.branch2[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4e.branch3[0].conv.weight.data = model.inception4e.branch3[0].conv.weight[:, new_indxIncp2, :, :]
model.inception4e.branch4[1].conv.weight.data = model.inception4e.branch4[1].conv.weight[:, new_indxIncp2, :, :]

Switching before sorting 207582.0
Switching after sorting 191527.0
Percentage of reduction 7.734292954109701 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 440.0
Switching after sorting 421.0
Percentage of reduction 4.318181818181818 %
Switching before sorting 447.0
Switching after sorting 448.0
Percentage of reduction -0.22371364653243847 %
Switching before sorting 335.0
Switching after sorting 331.0
Percentage of reduction 1.1940298507462686 %
Switching before sorting 276674.0
Switching after sorting 251977.0
Percentage of reduction 8.926389902918237 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 460.0
Switching after sorting 478.0
Percentage of reduction -3.9130434782608696 %
Switching before sorting 494.0
Switching after sorting 484.0
Percentage of reduction 2.0242914979757085 %
Switching before sorting 422.0
Switching after sorting 383.0
Percentage 

In [111]:
## Inception 7
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception4e.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception4e.branch1.conv.weight)
model.inception4e.branch1.conv.weight.data, _, sw_wt[6][0], sw_wt_sorted[6][0]  = compare_sw_sort(model.inception4e.branch1.conv.weight.data, new_indxb1, -3)
model.inception4e.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception4e.branch1.bn.weight.data, new_indxb1, -1)
model.inception4e.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception4e.branch1.bn.bias.data, new_indxb1, -1)
model.inception4e.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4e.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception4e.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4e.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception4e.branch2[0].conv.weight)
model.inception4e.branch2[0].conv.weight.data, _, sw_wt[6][1], sw_wt_sorted[6][1]  = compare_sw_sort(model.inception4e.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception4e.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4e.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception4e.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4e.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception4e.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4e.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception4e.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4e.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception4e.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception4e.branch2[1].conv.weight)
model.inception4e.branch2[1].conv.weight.data, _, sw_wt[6][2], sw_wt_sorted[6][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception4e.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4e.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception4e.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4e.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception4e.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4e.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception4e.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4e.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception4e.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception4e.branch3[0].conv.weight)
model.inception4e.branch3[0].conv.weight.data, _, sw_wt[6][3], sw_wt_sorted[6][3]  = compare_sw_sort(model.inception4e.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception4e.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4e.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception4e.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4e.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception4e.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4e.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception4e.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4e.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception4e.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception4e.branch3[1].conv.weight)
model.inception4e.branch3[1].conv.weight.data, _, sw_wt[6][4], sw_wt_sorted[6][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception4e.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4e.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception4e.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4e.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception4e.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4e.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception4e.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4e.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception4e.branch1.conv.weight.shape[0] + model.inception4e.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception4e.branch4[1].conv.weight)
model.inception4e.branch4[1].conv.weight.data, _, sw_wt[6][5], sw_wt_sorted[6][5]  = compare_sw_sort(model.inception4e.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception4e.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception4e.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception4e.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception4e.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception4e.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception4e.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception4e.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception4e.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception4e.branch1.conv.weight.shape[0] + model.inception4e.branch2[1].conv.weight.shape[0] + model.inception4e.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate

# Rearranging all branchs of inception 2 along channel dimension to correct Branch 1 sortng 
model.inception5a.branch1.conv.weight.data = model.inception5a.branch1.conv.weight[:, new_indxIncp2, :, :]
model.inception5a.branch2[0].conv.weight.data =model.inception5a.branch2[0].conv.weight[:, new_indxIncp2, :, :]
model.inception5a.branch3[0].conv.weight.data = model.inception5a.branch3[0].conv.weight[:, new_indxIncp2, :, :]
model.inception5a.branch4[1].conv.weight.data = model.inception5a.branch4[1].conv.weight[:, new_indxIncp2, :, :]

Switching before sorting 505420.0
Switching after sorting 458379.0
Percentage of reduction 9.307308772901745 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 860.0
Switching after sorting 849.0
Percentage of reduction 1.2790697674418605 %
Switching before sorting 838.0
Switching after sorting 794.0
Percentage of reduction 5.250596658711217 %
Switching before sorting 608.0
Switching after sorting 594.0
Percentage of reduction 2.3026315789473686 %
Switching before sorting 322037.0
Switching after sorting 295558.0
Percentage of reduction 8.22234712160404 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 500.0
Switching after sorting 514.0
Percentage of reduction -2.8 %
Switching before sorting 558.0
Switching after sorting 544.0
Percentage of reduction 2.5089605734767026 %
Switching before sorting 415.0
Switching after sorting 445.0
Percentage of reduction -7.22

In [112]:
## Inception 8
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception5a.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception5a.branch1.conv.weight)
model.inception5a.branch1.conv.weight.data, _, sw_wt[7][0], sw_wt_sorted[7][0]  = compare_sw_sort(model.inception5a.branch1.conv.weight.data, new_indxb1, -3)
model.inception5a.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception5a.branch1.bn.weight.data, new_indxb1, -1)
model.inception5a.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception5a.branch1.bn.bias.data, new_indxb1, -1)
model.inception5a.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5a.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception5a.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5a.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception5a.branch2[0].conv.weight)
model.inception5a.branch2[0].conv.weight.data, _, sw_wt[7][1], sw_wt_sorted[7][1]  = compare_sw_sort(model.inception5a.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception5a.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5a.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception5a.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5a.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception5a.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5a.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception5a.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5a.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception5a.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception5a.branch2[1].conv.weight)
model.inception5a.branch2[1].conv.weight.data, _, sw_wt[7][2], sw_wt_sorted[7][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception5a.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5a.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception5a.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5a.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception5a.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5a.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception5a.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5a.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception5a.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception5a.branch3[0].conv.weight)
model.inception5a.branch3[0].conv.weight.data, _, sw_wt[7][3], sw_wt_sorted[7][3]  = compare_sw_sort(model.inception5a.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception5a.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5a.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception5a.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5a.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception5a.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5a.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception5a.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5a.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception5a.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception5a.branch3[1].conv.weight)
model.inception5a.branch3[1].conv.weight.data, _, sw_wt[7][4], sw_wt_sorted[7][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception5a.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5a.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception5a.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5a.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception5a.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5a.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception5a.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5a.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception5a.branch1.conv.weight.shape[0] + model.inception5a.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception5a.branch4[1].conv.weight)
model.inception5a.branch4[1].conv.weight.data, _, sw_wt[7][5], sw_wt_sorted[7][5]  = compare_sw_sort(model.inception5a.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception5a.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5a.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception5a.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5a.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception5a.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5a.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception5a.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5a.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception5a.branch1.conv.weight.shape[0] + model.inception5a.branch2[1].conv.weight.shape[0] + model.inception5a.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate


# Rearranging all branchs of inception 2 along channel dimension to correct Branch 1 sortng 
model.inception5b.branch1.conv.weight.data = model.inception5b.branch1.conv.weight[:, new_indxIncp2, :, :]
model.inception5b.branch2[0].conv.weight.data =model.inception5b.branch2[0].conv.weight[:, new_indxIncp2, :, :]
model.inception5b.branch3[0].conv.weight.data = model.inception5b.branch3[0].conv.weight[:, new_indxIncp2, :, :]
model.inception5b.branch4[1].conv.weight.data = model.inception5b.branch4[1].conv.weight[:, new_indxIncp2, :, :]

Switching before sorting 785122.0
Switching after sorting 728124.0
Percentage of reduction 7.259763450775803 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 885.0
Switching after sorting 880.0
Percentage of reduction 0.5649717514124294 %
Switching before sorting 660.0
Switching after sorting 711.0
Percentage of reduction -7.7272727272727275 %
Switching before sorting 466.0
Switching after sorting 471.0
Percentage of reduction -1.0729613733905579 %
Switching before sorting 498903.0
Switching after sorting 465094.0
Percentage of reduction 6.776668009613091 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 503.0
Switching after sorting 461.0
Percentage of reduction 8.34990059642147 %
Switching before sorting 523.0
Switching after sorting 541.0
Percentage of reduction -3.4416826003824093 %
Switching before sorting 475.0
Switching after sorting 453.0
Percentage o

In [113]:
## Inception 9
# Branch 1
# conv1_weight_reshaped, new_indx1 = sortFullMatrix(np.asarray(conv1_weight_reshaped.to('cpu').detach().numpy()))
# new_indxb1 = torch.randperm(model.inception5b.branch1.conv.weight.shape[0])
_, new_indxb1 = sortFullMatrix_V2(model.inception5b.branch1.conv.weight)
model.inception5b.branch1.conv.weight.data, _, sw_wt[8][0], sw_wt_sorted[8][0]  = compare_sw_sort(model.inception5b.branch1.conv.weight.data, new_indxb1, -3)
model.inception5b.branch1.bn.weight.data, _, _, _  = compare_sw_sort(model.inception5b.branch1.bn.weight.data, new_indxb1, -1)
model.inception5b.branch1.bn.bias.data, _, _, _  = compare_sw_sort(model.inception5b.branch1.bn.bias.data, new_indxb1, -1)
model.inception5b.branch1.bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5b.branch1.bn.running_mean.data, new_indxb1, -1)
model.inception5b.branch1.bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5b.branch1.bn.running_var.data, new_indxb1, -1)

# Branch 2
# conv 1x1
_, new_indxb2a = sortFullMatrix_V2(model.inception5b.branch2[0].conv.weight)
model.inception5b.branch2[0].conv.weight.data, _, sw_wt[8][1], sw_wt_sorted[8][1]  = compare_sw_sort(model.inception5b.branch2[0].conv.weight.data, new_indxb2a, -3)
model.inception5b.branch2[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5b.branch2[0].bn.weight.data, new_indxb2a, -1)
model.inception5b.branch2[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5b.branch2[0].bn.bias.data, new_indxb2a, -1)
model.inception5b.branch2[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5b.branch2[0].bn.running_mean.data, new_indxb2a, -1)
model.inception5b.branch2[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5b.branch2[0].bn.running_var.data, new_indxb2a, -1)

# conv 3x3
new_b2conv2_weight = model.inception5b.branch2[1].conv.weight[:, new_indxb2a, :, :]  # Re-arranging along channel
_, new_indxb2b = sortFullMatrix_V2(model.inception5b.branch2[1].conv.weight)
model.inception5b.branch2[1].conv.weight.data, _, sw_wt[8][2], sw_wt_sorted[8][2]  = compare_sw_sort(new_b2conv2_weight.data, new_indxb2b, -3)
model.inception5b.branch2[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5b.branch2[1].bn.weight.data, new_indxb2b, -1)
model.inception5b.branch2[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5b.branch2[1].bn.bias.data, new_indxb2b, -1)
model.inception5b.branch2[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5b.branch2[1].bn.running_mean.data, new_indxb2b, -1)
model.inception5b.branch2[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5b.branch2[1].bn.running_var.data, new_indxb2b, -1)

new_indxb2b = new_indxb2b + model.inception5b.branch1.conv.weight.shape[0] # incrementing all indeces  b2b for concatenation at the second layer

# Branch 3
# conv 1x1
_, new_indxb3a = sortFullMatrix_V2(model.inception5b.branch3[0].conv.weight)
model.inception5b.branch3[0].conv.weight.data, _, sw_wt[8][3], sw_wt_sorted[8][3]  = compare_sw_sort(model.inception5b.branch3[0].conv.weight.data, new_indxb3a, -3)
model.inception5b.branch3[0].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5b.branch3[0].bn.weight.data, new_indxb3a, -1)
model.inception5b.branch3[0].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5b.branch3[0].bn.bias.data, new_indxb3a, -1)
model.inception5b.branch3[0].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5b.branch3[0].bn.running_mean.data, new_indxb3a, -1)
model.inception5b.branch3[0].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5b.branch3[0].bn.running_var.data, new_indxb3a, -1)

# conv 3x3
new_b3conv2_weight = model.inception5b.branch3[1].conv.weight[:, new_indxb3a, :, :]  
_, new_indxb3b = sortFullMatrix_V2(model.inception5b.branch3[1].conv.weight)
model.inception5b.branch3[1].conv.weight.data, _, sw_wt[8][4], sw_wt_sorted[8][4]  = compare_sw_sort(new_b3conv2_weight.data, new_indxb3b, -3)
model.inception5b.branch3[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5b.branch3[1].bn.weight.data, new_indxb3b, -1)
model.inception5b.branch3[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5b.branch3[1].bn.bias.data, new_indxb3b, -1)
model.inception5b.branch3[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5b.branch3[1].bn.running_mean.data, new_indxb3b, -1)
model.inception5b.branch3[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5b.branch3[1].bn.running_var.data, new_indxb3b, -1)
new_indxb3b = new_indxb3b + model.inception5b.branch1.conv.weight.shape[0] + model.inception5b.branch2[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

# Branch 4
_, new_indxb4 = sortFullMatrix_V2(model.inception5b.branch4[1].conv.weight)
model.inception5b.branch4[1].conv.weight.data, _, sw_wt[8][5], sw_wt_sorted[8][5]  = compare_sw_sort(model.inception5b.branch4[1].conv.weight.data, new_indxb4, -3)
model.inception5b.branch4[1].bn.weight.data, _, _, _  = compare_sw_sort(model.inception5b.branch4[1].bn.weight.data, new_indxb4, -1)
model.inception5b.branch4[1].bn.bias.data, _, _, _  = compare_sw_sort(model.inception5b.branch4[1].bn.bias.data, new_indxb4, -1)
model.inception5b.branch4[1].bn.running_mean.data, _, _, _  = compare_sw_sort(model.inception5b.branch4[1].bn.running_mean.data, new_indxb4, -1)
model.inception5b.branch4[1].bn.running_var.data, _, _, _  = compare_sw_sort(model.inception5b.branch4[1].bn.running_var.data, new_indxb4, -1)

new_indxb4 = new_indxb4 + model.inception5b.branch1.conv.weight.shape[0] + model.inception5b.branch2[1].conv.weight.shape[0] + model.inception5b.branch3[1].conv.weight.shape[0] # incrementing all indeces  b3b for concatenation at the second layer

new_indxIncp2 = np.concatenate((new_indxb1, new_indxb2b, new_indxb3b, new_indxb4), 0) #np.concatenate


# Rearranging all branchs of fc
model.fc.weight.data = model.fc.weight[:, new_indxIncp2] 


Switching before sorting 974141.0
Switching after sorting 881309.0
Percentage of reduction 9.529626614627656 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 763.0
Switching after sorting 736.0
Percentage of reduction 3.5386631716906947 %
Switching before sorting 760.0
Switching after sorting 779.0
Percentage of reduction -2.5 %
Switching before sorting 117.0
Switching after sorting 107.0
Percentage of reduction 8.547008547008547 %
Switching before sorting 612600.0
Switching after sorting 572582.0
Percentage of reduction 6.532484492327783 %
Switching before sorting 0.0
Switching after sorting 0.0
Percentage of reduction nan %
Switching before sorting 602.0
Switching after sorting 614.0
Percentage of reduction -1.9933554817275747 %
Switching before sorting 574.0
Switching after sorting 587.0
Percentage of reduction -2.264808362369338 %
Switching before sorting 584.0
Switching after sorting 581.0
Percentage of reduction 0.5

In [127]:
sw_wtConv = np.array([ 35957,  15587, 415246])
sw_wt_sortedConv = np.array([ 31893,  13101, 369064])

In [114]:
print('Switching before sorting', sw_wt) 
print('Switching after sorting', sw_wt_sorted)

Switching before sorting [[  46409.   71234.  401397.   11079.   16162.   22585.]
 [ 122703.  122718.  791884.   30178.  101265.   60246.]
 [ 339103.  166546.  642889.   25458.   25496.  114067.]
 [ 301081.  213995.  788117.   43035.   49658.  121797.]
 [ 240730.  249343. 1047734.   41723.   49590.  122820.]
 [ 207582.  276674. 1314867.   58815.   67032.  121556.]
 [ 505420.  322037. 1601572.   60805.  135035.  254589.]
 [ 785122.  498903. 1601072.   96112.  136782.  394429.]
 [ 974141.  612600. 1490417.  148579.  132633.  375670.]]
Switching after sorting [[  41392.   62260.  367114.    9904.   14254.   20167.]
 [ 107634.  108056.  725872.   27190.   89470.   53876.]
 [ 304019.  151218.  586956.   23240.   22491.  104420.]
 [ 276999.  192303.  723919.   40584.   44881.  112794.]
 [ 222237.  223067.  958951.   39267.   43146.  113180.]
 [ 191527.  251977. 1209593.   53092.   59110.  111817.]
 [ 458379.  295558. 1481897.   56542.  117060.  234146.]
 [ 728124.  465094. 1469553.   91389. 

In [128]:
print('Percentage of reduction in switching across the first conv blocks', (sw_wtConv-sw_wt_sortedConv)/sw_wtConv)
print('Percentage of reduction in switching across all inception modules', (np.sum(sw_wt, 0)-np.sum(sw_wt_sorted, 0))/np.sum(sw_wt, 0))

Percentage of reduction in switching across all inception modules [0.11302389 0.15949188 0.111216  ]
Percentage of reduction in switching across all inception modules [0.0882014  0.08363489 0.08930584 0.06520753 0.14703925 0.07414286]


In [115]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
filename = 'dog.jpg'
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0].shape)
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities.shape)

torch.Size([1000])
torch.Size([1000])


In [116]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.9378383755683899
Pomeranian 0.008283410221338272
Great Pyrenees 0.005603067111223936
Arctic fox 0.005527761299163103
white wolf 0.004741047043353319


In [3]:
# Download an example image from the pytorch website
# import urllib
# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# try: urllib.URLopener().retrieve(url, filename)
# except: urllib.request.urlretrieve(url, filename)

In [103]:
# Download ImageNet labels
# !wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt